In [ ]:
try:
    import google.colab
    # We're in Colab
    !pip install git+https://github.com/landoskape/syd.git

except ImportError:
    # Not in Colab
    print("This notebook is designed for Google Colab.")
    print("To install locally, run: pip install syd")

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from syd import make_viewer

# Create a simple dataset
T = 1000
amplitudes = list(np.linspace(1, 5.0, 100))
frequencies = list(np.linspace(1, 20.0, 100))
num_amplitudes = len(amplitudes)
num_frequencies = len(frequencies)

# Create a time array and an empty numpy array to store the data
t = np.linspace(0, 1, T)
data = np.zeros((num_amplitudes, num_frequencies, T))

# Fill the data array with the sine wave data
for i, amplitude in enumerate(amplitudes):
    for j, frequency in enumerate(frequencies):
        data[i, j, :] = np.sin(frequency * t * 2 * np.pi) * amplitude

# Create a dataset dictionary
dataset = {
    "amplitudes": amplitudes,
    "frequencies": frequencies,
    "t": t,
    "data": data,
}

# Now, we have a simple dataset to use. Let's pretend that it's a complicated and large dataset
# that you've stored on disk. You'll probably have a nice function that loads the data. 
def load_data(amplitude, frequency):
    """Load the data for a given amplitude and frequency."""
    iamplitude = dataset["amplitudes"].index(amplitude)
    ifrequency = dataset["frequencies"].index(frequency)
    return dataset["t"], dataset["data"][iamplitude, ifrequency]

num_amplitudes = len(dataset["amplitudes"])
num_frequencies = len(dataset["frequencies"]) 

# Make the viewer and add all the components
viewer = make_viewer()
viewer.add_integer("amplitude", min=0, max=num_amplitudes-1)
viewer.add_integer("frequency", min=0, max=num_frequencies-1)
viewer.add_selection("color", value="red", options=["red", "blue", "green"])
viewer.add_float("horizontal", value=0.0, min=-10.0, max=10.0)
viewer.add_unbounded_float("offset", value=0.0)
viewer.add_float_range("xlimits", value=(0.0, 1.0), min=-10.0, max=1.0)
viewer.add_float_range("ylimits", value=(-1, 1), min=-10, max=10)
viewer.add_multiple_selection("labels", value=["x", "y", "title", "legend"], options=["x", "y", "title", "legend"])
viewer.add_boolean("show_grid", value=True)

def plot(state):
    # Get the data based on the current state
    current_amplitude = dataset["amplitudes"][state["amplitude"]]
    current_frequency = dataset["frequencies"][state["frequency"]]
    time, data = load_data(current_amplitude, current_frequency)

    # Get all the other parameters for plotting
    color = state["color"]
    horizontal = state["horizontal"]
    offset = state["offset"]
    xlimits = state["xlimits"]
    ylimits = state["ylimits"]
    labels = state["labels"]
    show_grid = state["show_grid"]

    # Make your figure
    fig, ax = plt.subplots(1, 1, figsize=(5, 4), layout="constrained")
    ax.plot(time, data + offset, color=color, label="data")
    ax.axhline(horizontal, color="black", linestyle="--", label="horizontal")
    ax.set_xlim(xlimits)
    ax.set_ylim(ylimits)
    if "x" in labels:
        ax.set_xlabel("Time")
    if "y" in labels:
        ax.set_ylabel("Amplitude")
    if "title" in labels:
        ax.set_title("Sine Wave")
    if "legend" in labels:
        ax.legend(loc="best")
    if show_grid:
        ax.grid()
    return fig

# Tell the viewer to use your plot method
viewer.set_plot(plot)

# Create a callback function to change the y-limits and horizontal line based on the offset
def update_offset(state):
    # Get the data based on the current state
    current_amplitude = dataset["amplitudes"][state["amplitude"]]
    current_frequency = dataset["frequencies"][state["frequency"]]
    data = load_data(current_amplitude, current_frequency)[1]

    # Get the offset
    offset = state["offset"]
    min_plot_data = np.min(data + offset)
    max_plot_data = np.max(data + offset)

    # Update the min/max values of the horizontal line and the y-limits to match the plot data
    viewer.update_float_range("ylimits", value=(min_plot_data, max_plot_data), min=min_plot_data, max=max_plot_data)
    viewer.update_float("horizontal", value=offset, min=min_plot_data, max=max_plot_data)

# Add the callback to the viewer
viewer.on_change(["amplitude", "offset"], update_offset)

# Show the viewer!
# viewer.show() # for viewing in a jupyter notebook
viewer.share() # for viewing in a web browser

Starting Flask server...
 * Syd Flask server running on http://127.0.0.1:5000
